# Dinosaur Wikipedia File Extractions
This file completes the data acquisition section of Homework 1 using [Wikimedia REST API](https://www.mediawiki.org/wiki/Wikimedia_REST_API). This method will request monthly counts of page views for all dinosaur articles in the dinosaur_genera.cleaned.SEPT.2022.csv file. The API documentation, [pageviews/per-article](https://wikimedia.org/api/rest_v1/#/Pageviews%20data), covers additional details about the data.

## Crediting Code

Some code used in this file uses the [example notebook](https://drive.google.com/file/d/1gtFZAjRoOShsqZKuNhiiSn9Ko4ky-CSC/view?usp=sharing) provided by professor McDonald. It is [licensed CC0](https://creativecommons.org/share-your-work/public-domain/cc0/) and is allowed to be reused.


In [1]:
# These are standard python modules
import json, time, urllib.parse

# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests
import pandas as pd

The example relies on some constants that help make the code a bit more readable. It will also pull data in from the dinosaur_genera.cleaned.SEPT.2022.csv to be used in the constants as well.

In [2]:
# Getting the dinosaur names for the data pull
wiki_dinos = pd.read_csv("dinosaur_genera.cleaned.SEPT.2022.csv")
# issues with url.parse.quote handling the original entry from the csv
wiki_dinos['name'][0] = '"Coelosaurus" antiquus'

In [3]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include a "unique ID" that will allow them to
# contact you if something happens - such as - your code exceeding request limits - or some other error happens
REQUEST_HEADERS = {
    'User-Agent': '<rafrisci@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2022',
}

# Here we get the dinosaur names for the Wikipedia pages
ARTICLE_TITLES = wiki_dinos['name']

# For the different accesses needed
ACCESS_TYPE = ['mobile-web', 'mobile-app', 'desktop']

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-app",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015070100",
    "end":         "2022093000"    
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [4]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):
    # Make sure we have an article title
    if not article_title: return None
    
    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(article_title.replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


Now we will pull the data for all dinosaurs for all mobile and desktop access.

The range of data we are pulling for is from July 2015 to September 2022.

In [5]:
# Setting up empty lists to store the different accesses of data
mobile_web_viewing = []
mobile_app_viewing = []
desktop_viewing = []
# Retrieving the data for all access types and articles
for access in ACCESS_TYPE:
    ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE["access"] = access
    for dinosaur in ARTICLE_TITLES:
        print("Getting pageview data for: ", access, dinosaur)
        page = request_pageviews_per_article(dinosaur)
        if access == "mobile-web":
            mobile_web_viewing = mobile_web_viewing + page['items']
        elif access == "mobile-app":
            mobile_app_viewing = mobile_app_viewing + page['items']
        else:
            desktop_viewing = desktop_viewing + page['items']

Getting pageview data for:  mobile-web "Coelosaurus" antiquus
Getting pageview data for:  mobile-web Aachenosaurus
Getting pageview data for:  mobile-web Aardonyx
Getting pageview data for:  mobile-web Abdarainurus
Getting pageview data for:  mobile-web Abditosaurus
Getting pageview data for:  mobile-web Abelisaurus
Getting pageview data for:  mobile-web Abrictosaurus
Getting pageview data for:  mobile-web Abrosaurus
Getting pageview data for:  mobile-web Abydosaurus
Getting pageview data for:  mobile-web Acantholipan
Getting pageview data for:  mobile-web Acanthopholis
Getting pageview data for:  mobile-web Achelousaurus
Getting pageview data for:  mobile-web Acheroraptor
Getting pageview data for:  mobile-web Achillesaurus
Getting pageview data for:  mobile-web Achillobator
Getting pageview data for:  mobile-web Acristavus
Getting pageview data for:  mobile-web Acrocanthosaurus
Getting pageview data for:  mobile-web Acrotholus
Getting pageview data for:  mobile-web Actiosaurus
Gettin

Getting pageview data for:  mobile-web Austrosaurus
Getting pageview data for:  mobile-web Avaceratops
Getting pageview data for:  mobile-web Avalonianus
Getting pageview data for:  mobile-web Avebrevicauda
Getting pageview data for:  mobile-web Avemetatarsalia
Getting pageview data for:  mobile-web Aviatyrannis
Getting pageview data for:  mobile-web Avimimus
Getting pageview data for:  mobile-web Avipes
Getting pageview data for:  mobile-web Avisaurus
Getting pageview data for:  mobile-web Azendohsaurus
Getting pageview data for:  mobile-web Azhdarchoidea
Getting pageview data for:  mobile-web Baalsaurus
Getting pageview data for:  mobile-web Bactrosaurus
Getting pageview data for:  mobile-web Bagaceratops
Getting pageview data for:  mobile-web Bagaraatan
Getting pageview data for:  mobile-web Bagualia
Getting pageview data for:  mobile-web Bagualosaurus
Getting pageview data for:  mobile-web Bahariasaurus
Getting pageview data for:  mobile-web Bainoceratops
Getting pageview data for:

Getting pageview data for:  mobile-web Concavenator
Getting pageview data for:  mobile-web Conchoraptor
Getting pageview data for:  mobile-web Condorraptor
Getting pageview data for:  mobile-web Convolosaurus
Getting pageview data for:  mobile-web Coronosaurus
Getting pageview data for:  mobile-web Corythoraptor
Getting pageview data for:  mobile-web Corythosaurus
Getting pageview data for:  mobile-web Craspedodon
Getting pageview data for:  mobile-web Craterosaurus
Getting pageview data for:  mobile-web Crichtonpelta
Getting pageview data for:  mobile-web Crichtonsaurus
Getting pageview data for:  mobile-web Cristatusaurus
Getting pageview data for:  mobile-web Crittendenceratops
Getting pageview data for:  mobile-web Crocodilia
Getting pageview data for:  mobile-web Crosbysaurus
Getting pageview data for:  mobile-web Crurotarsi
Getting pageview data for:  mobile-web Cruxicheiros
Getting pageview data for:  mobile-web Cryolophosaurus
Getting pageview data for:  mobile-web Cryptosaurus

Getting pageview data for:  mobile-web Fylax
Getting pageview data for:  mobile-web Galeamopus
Getting pageview data for:  mobile-web Galesaurus
Getting pageview data for:  mobile-web Galleonosaurus
Getting pageview data for:  mobile-web Gallimimus
Getting pageview data for:  mobile-web Galvesaurus
Getting pageview data for:  mobile-web Gannansaurus
Getting pageview data for:  mobile-web Ganzhousaurus
Getting pageview data for:  mobile-web Gargoyleosaurus
Getting pageview data for:  mobile-web Garrigatitan
Getting pageview data for:  mobile-web Garudimimus
Getting pageview data for:  mobile-web Gasosaurus
Getting pageview data for:  mobile-web Gasparinisaura
Getting pageview data for:  mobile-web Gastonia (dinosaur)
Getting pageview data for:  mobile-web Geminiraptor
Getting pageview data for:  mobile-web Genusaurus
Getting pageview data for:  mobile-web Genyodectes
Getting pageview data for:  mobile-web Geosaurus
Getting pageview data for:  mobile-web Geranosaurus
Getting pageview dat

Getting pageview data for:  mobile-web Jobaria
Getting pageview data for:  mobile-web Jubbulpuria
Getting pageview data for:  mobile-web Judiceratops
Getting pageview data for:  mobile-web Juratyrant
Getting pageview data for:  mobile-web Juravenator
Getting pageview data for:  mobile-web Kaatedocus
Getting pageview data for:  mobile-web Kaijiangosaurus
Getting pageview data for:  mobile-web Kaijutitan
Getting pageview data for:  mobile-web Kakuru
Getting pageview data for:  mobile-web Kamuysaurus
Getting pageview data for:  mobile-web Kangnasaurus
Getting pageview data for:  mobile-web Kansaignathus
Getting pageview data for:  mobile-web Karongasaurus
Getting pageview data for:  mobile-web Katepensaurus
Getting pageview data for:  mobile-web Kayentavenator
Getting pageview data for:  mobile-web Kazaklambia
Getting pageview data for:  mobile-web Kelmayisaurus
Getting pageview data for:  mobile-web Kelumapusaura
Getting pageview data for:  mobile-web Kemkemia
Getting pageview data for: 

...

Getting pageview data for:  desktop Kuszholia
Getting pageview data for:  desktop Kwanasaurus
Getting pageview data for:  desktop Labocania
Getting pageview data for:  desktop Laevisuchus
Getting pageview data for:  desktop Lagerpeton
Getting pageview data for:  desktop Lagosuchus
Getting pageview data for:  desktop Laiyangosaurus
Getting pageview data for:  desktop Lajasvenator
Getting pageview data for:  desktop Lambeosaurus
Getting pageview data for:  desktop Lametasaurus
Getting pageview data for:  desktop Lamplughsaura
Getting pageview data for:  desktop Lanzhousaurus
Getting pageview data for:  desktop Laosaurus
Getting pageview data for:  desktop Lapampasaurus
Getting pageview data for:  desktop Laplatasaurus
Getting pageview data for:  desktop Lapparentosaurus
Getting pageview data for:  desktop Laquintasaura
Getting pageview data for:  desktop Lariosaurus
Getting pageview data for:  desktop Latenivenatrix
Getting pageview data for:  desktop Latirhinus
Getting pageview data for

Getting pageview data for:  desktop Nasutoceratops
Getting pageview data for:  desktop Navajoceratops
Getting pageview data for:  desktop Nebulasaurus
Getting pageview data for:  desktop Nedcolbertia
Getting pageview data for:  desktop Nedoceratops
Getting pageview data for:  desktop Neimongosaurus
Getting pageview data for:  desktop Nemegtomaia
Getting pageview data for:  desktop Nemegtonykus
Getting pageview data for:  desktop Nemegtosaurus
Getting pageview data for:  desktop Neosodon
Getting pageview data for:  desktop Neovenator
Getting pageview data for:  desktop Neuquenraptor
Getting pageview data for:  desktop Neuquensaurus
Getting pageview data for:  desktop Ngwevu
Getting pageview data for:  desktop Nhandumirim
Getting pageview data for:  desktop Niebla antiqua
Getting pageview data for:  desktop Nigersaurus
Getting pageview data for:  desktop Ningyuansaurus
Getting pageview data for:  desktop Ninjatitan
Getting pageview data for:  desktop Niobrarasaurus
Getting pageview data 

Getting pageview data for:  desktop Protecovasaurus
Getting pageview data for:  desktop Protoavis
Getting pageview data for:  desktop Protoceratops
Getting pageview data for:  desktop Protognathosaurus
Getting pageview data for:  desktop Protohadros
Getting pageview data for:  desktop Protorosauria
Getting pageview data for:  desktop Protorosaurus
Getting pageview data for:  desktop Pseudosuchia
Getting pageview data for:  desktop Psittacosaurus
Getting pageview data for:  desktop Pteropelyx
Getting pageview data for:  desktop Pterosaur
Getting pageview data for:  desktop Pterospondylus
Getting pageview data for:  desktop Puertasaurus
Getting pageview data for:  desktop Pukyongosaurus
Getting pageview data for:  desktop Pulanesaura
Getting pageview data for:  desktop Punatitan
Getting pageview data for:  desktop Pycnonemosaurus
Getting pageview data for:  desktop Pyroraptor
Getting pageview data for:  desktop Qantassaurus
Getting pageview data for:  desktop Qianzhousaurus
Getting pagev

Getting pageview data for:  desktop Spondylosoma
Getting pageview data for:  desktop Squalodon
Getting pageview data for:  desktop Staurikosaurus
Getting pageview data for:  desktop Stegoceras
Getting pageview data for:  desktop Stegopelta
Getting pageview data for:  desktop Stegosaurides
Getting pageview data for:  desktop Stegosaurus
Getting pageview data for:  desktop Stegouros
Getting pageview data for:  desktop Stellasaurus
Getting pageview data for:  desktop Stenonychosaurus
Getting pageview data for:  desktop Stenopelix
Getting pageview data for:  desktop Stephanosaurus
Getting pageview data for:  desktop Stokesosaurus
Getting pageview data for:  desktop Streptospondylus
Getting pageview data for:  desktop Struthiomimus
Getting pageview data for:  desktop Struthiosaurus
Getting pageview data for:  desktop Styracosaurus
Getting pageview data for:  desktop Succinodon
Getting pageview data for:  desktop Suchomimus
Getting pageview data for:  desktop Suchoprion
Getting pageview data

Getting pageview data for:  desktop Wulatelong
Getting pageview data for:  desktop Wulong bohaiensis
Getting pageview data for:  desktop Wyleyia
Getting pageview data for:  desktop Xenoceratops
Getting pageview data for:  desktop Xenoposeidon
Getting pageview data for:  desktop Xenotarsosaurus
Getting pageview data for:  desktop Xianshanosaurus
Getting pageview data for:  desktop Xiaosaurus
Getting pageview data for:  desktop Xiaotingia
Getting pageview data for:  desktop Xingtianosaurus
Getting pageview data for:  desktop Xingxiulong
Getting pageview data for:  desktop Xinjiangovenator
Getting pageview data for:  desktop Xinjiangtitan
Getting pageview data for:  desktop Xiongguanlong
Getting pageview data for:  desktop Xixianykus
Getting pageview data for:  desktop Xixiasaurus
Getting pageview data for:  desktop Xixiposaurus
Getting pageview data for:  desktop Xiyunykus
Getting pageview data for:  desktop Xuanhanosaurus
Getting pageview data for:  desktop Xuanhuaceratops
Getting pagev

We can now start creating the json files desired using pandas. First we set all traffic data pulled as a pandas DataFrame.

In [6]:
# Making the dict lists into pandas DFs
desktop = pd.DataFrame(desktop_viewing)
mobile_web = pd.DataFrame(mobile_web_viewing)
mobile_app = pd.DataFrame(mobile_app_viewing)

Making the monthly desktop file only requires exporting the desktop data.

In [9]:
desk_export = desktop
desk_export['timestamp'] = pd.to_datetime(desk_export['timestamp'], format = '%Y%m%d%H')
desk_export = desk_export.drop(['access'], axis = 1)
desk_export = desk_export.sort_values(by='article')
desk_export.to_json(r'data/dino_monthly_desktop_201507-202209.json')

Making the monthly mobile file requires joining both mobile-web and mobile-app data and summing the views for rows with the same timestamp and article.

In [15]:
# Combine mobile traffics and sum the views for the same article and month
mobile_joined = pd.concat([mobile_web, mobile_app])
mobile_joined = mobile_joined.groupby(['project', 'article', 'granularity', 'timestamp', 'agent']).sum('views')
mobile_joined = mobile_joined.reset_index()
# Create json file
mobile_joined['timestamp'] = pd.to_datetime(mobile_joined['timestamp'], format = '%Y%m%d%H')
mobile_joined = mobile_joined.sort_values(by='article')
mobile_joined.to_json(r'data/dino_monthly_mobile_201507-202209.json')

Making the monthly cumulative file requires summing the views for all traffics in a similar manner when making the monthly mobile file, then cumulative summing by article so the views are for the article's timestamp and all timestamps previous to that one.

In [16]:
# Combine all traffics and get cumulative sum for every article's month and all months preceding it
cumulative = pd.concat([mobile_web, mobile_app, desktop])
cumulative = cumulative.groupby(['project', 'article', 'granularity', 'timestamp', 'agent']).sum('views')
cumulative = cumulative.reset_index()
cumulative = cumulative.sort_values(by = ['article', 'timestamp'], ascending = True)
cumulative['cumsum_views'] = cumulative[['article', 'views']].groupby('article').cumsum()
# Get rid of the previous views column
cumulative = cumulative.drop('views', axis = 1)
cumulative.rename(columns={'cumsum_views': 'views'}, inplace=True)
cumulative = cumulative.sort_values(by='article')
# Create json file
cumulative.to_json(r'data/dino_monthly_cumulative_201507-202209.json')